In [11]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import WebDriverException
from IPython.display import clear_output

In [2]:
TIEMPO_DE_ESPERA = 5.5

start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

def createDriver():
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--window-size=1625x2444")
    # options.headless = True
    return webdriver.Chrome(options = chrome_options)

In [3]:
#browser.quit()

In [4]:
#browser.get('https://www.forever21.com/us/shop')

In [5]:
url_base = "https://www.forever21.com"

In [245]:
browser.save_screenshot('asd.png')

True

In [6]:
ipath = 'LINKS_F21.xlsx'
LINKS = pd.read_excel(ipath)

In [7]:
LINKS = LINKS["MEN"].tolist()

In [8]:
def asignarMoneda(driver):
    time.sleep(.6)
    #CLICK EN LA PARTE SUPERIOR DERECHA, DONDE HAY UN MODAL DE CONFIGURACIÓN DE PAÍS / MONEDA
    driver.execute_script("document.getElementsByClassName('table sameline fr')[0].getElementsByTagName('span')[0].click();")
    #ASIGNAR DOLAR 
    driver.execute_script("document.getElementsByTagName('select')[1].value ='USD'")
    time.sleep(2)
    #CLICK EN EL BOTON DE GUARDAR 
    driver.execute_script("document.getElementsByClassName('glControls')[0].getElementsByTagName('input')[0].click();")
    


In [9]:
lista_productos = []

In [13]:
browser = createDriver()
browser.get('https://www.forever21.com/us/shop')

In [14]:
for link in LINKS:
    try:
        browser.get(link)
    except WebDriverException:
        
        browser = createDriver()
        time.sleep(1)
        continue
    asignarMoneda(browser)
    tipologia = browser.find_element_by_xpath('//*[@id="multi_submenu_title"]/h1').text
    
    max_page = browser.execute_script('''
        var aux = document.getElementsByClassName('fr txr show_desktop s_page')[0].getElementsByClassName('pageno').length;

        var max_page = document.getElementsByClassName('fr txr show_desktop s_page')[0].getElementsByClassName('pageno')[aux-2].textContent;

        return max_page;

        ''')
    max_page = int(max_page)

    for page in range(max_page):

        #cantidad de productos dentro de la página.
    ##  productos = browser.find_element_by_id('products').find_elements_by_class_name('pi_container')
        
        browser.implicitly_wait(2)
        productos = browser.execute_script("var x = document.getElementsByClassName('qv square').length; return x;")
        if browser.current_url == 'https://www.forever21.com/us/shop':
                break
        for index_articulo in range(productos):
            try:
                #URL = articulo.find_element_by_tag_name('div').find_element_by_tag_name('div').find_element_by_tag_name('a').get_attribute('href')
                URL = url_base + browser.execute_script(f"var x = document.getElementsByClassName('item_slider product_link')[{index_articulo}].getAttribute('href');return x;")
            except:
                continue
        ##  ActionChains(browser).move_to_element(articulo).perform()
            try:
                browser.execute_script(f"document.getElementsByClassName('qv square')[{index_articulo}].click();")
            except:
                continue
            #boton.click()
            if browser.current_url == 'https://www.forever21.com/us/shop':
                break
            #ELEMENT_COLORS = browser.find_element_by_xpath('//*[@id="quickview_pop"]/div/div/div/div[2]/div[2]').find_element_by_id('colorButton').find_elements_by_tag_name('li')
            ###ELEMENT_COLORS = browser.execute_script(f"var x = document.getElementsByClassName('color')[{index_articulo}].getElementsByTagName('li').length; return x")
            browser.implicitly_wait(2)
            try:
                ELEMENT_COLORS = browser.execute_script(f"var x = document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li').length; return x;")
            except:
                time.sleep(3.5)
                pass
            try:
                ELEMENT_COLORS = browser.execute_script(f"var x = document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li').length; return x;")
            except:
                continue

            for item in range(ELEMENT_COLORS):

                #browser.execute_script(f"document.getElementsByClassName('color')[{index_articulo}].getElementsByTagName('li')[{item}].getElementsByTagName('span')[0].getElementsByTagName('img')[0].click();")
                time.sleep(.5)
                try:
                    browser.execute_script(f"document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li')[{item}].click();")
                except:
                    try:
                        browser.implicitly_wait(10)
                        browser.execute_script(f"document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li')[{item}].click();")
                    except:
                        break
                time.sleep(.5)
                
                try:
                    NOMBRE_COLOR = browser.execute_script("var x = document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByTagName('p')[0].textContent; return x;").strip()
                except:
                    browser.implicitly_wait(10)
                    NOMBRE_COLOR = browser.execute_script("var x = document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByTagName('p')[0].textContent; return x;").strip()

                #NOMBRE_COLOR = item.find_element_by_tag_name('span').find_element_by_tag_name('img').get_attribute('alt')
                #item.click()

                TALLES = browser.find_element_by_xpath('//*[@id="sizeButton"]').find_elements_by_tag_name('li')
                for TALLE in TALLES:

                    #TALLE BLOQUEADO ES OOS
                    if TALLE.find_element_by_tag_name('span').get_attribute('class') != "oos":

                        NOMBRE_TALLE = TALLE.find_element_by_tag_name('span').text

                        # IMG PRODUCTO
                        time.sleep(0.5)
                        try:
                            IMG = browser.execute_script(f"var x = document.getElementsByClassName('ws_100 owl-carousel owl-loaded owl-drag')[0].getElementsByTagName('img')[0].getAttribute('src'); return x")
                        except:
                            IMG = None
                        #IMG = browser.execute_script(f"var x = document.getElementsByClassName('product_image')[{index_articulo}].getAttribute('src'); return x")
                        # DESCRIPCION
                        DESCRIPCION = browser.execute_script(f"var x = document.getElementsByClassName('p_name')[{index_articulo}].textContent; return x")

                        # PRECIO
                        PRECIO = browser.execute_script("var x = document.getElementsByClassName('p_price pt_5 ws_100 row mb_10')[0].textContent; return x").strip()
                        # CODIGO
                        codigo = browser.execute_script("var x = document.getElementsByClassName('t_mid pt_10')[0].textContent; return x")
                        CODIGO = codigo.split()[-1]

                        lista_productos.append([CODIGO,
                                                NOMBRE_TALLE,
                                                NOMBRE_COLOR,
                                                'MEN',
                                                DESCRIPCION,
                                                PRECIO,
                                                IMG,
                                                URL,
                                                tipologia,
                                                page])

            try:
                browser.execute_script("document.getElementsByClassName('quick_close')[0].click();")
            except:
                pass
        #NEXTPAGE
        time.sleep(.8)
        try:
##            browser.execute_script("document.getElementsByClassName('p_next')[0].click();")
            #jquery click next page 
            browser.execute_script("$('#top-pager li span.p_next').click();")
            browser.implicitly_wait(15)
        except:
            #print(browser.current_url,
                # tipologia,
                # link,
                # page,
                # max_page)
            break

In [17]:
link

'https://www.forever21.com/us/shop/catalog/category/21men/mens-bottom-shorts'

In [18]:
LINKS

['https://www.forever21.com/us/shop/catalog/category/21men/mens-tees-tanks-graphic',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-bottom',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-jackets-and-coats',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-sweaters',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-sweatshirts-hoodies',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-bottom-swim',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-shirts',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-bottom-denim',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-bottom-shorts']

In [28]:
browser.save_screenshot('asd.png')

True

In [15]:
df = pd.DataFrame(lista_productos)

In [16]:
len(df.drop_duplicates())

3015

In [17]:
len(df)

3015

In [18]:
df.rename(columns={0:'sku',
                   1:'talle',
                   2:'color',
                   3:'sexo',
                   4:'descripcion',
                   5:'precio',
                   6:'url_jpg',
                   7:'url',
                   8:'categoria'},inplace=True)
#df.drop_duplicates()

In [19]:
df.rename(columns={9:'drop'},inplace=True)

In [20]:
df.drop('drop',axis=1,inplace=True)

In [21]:
df.drop_duplicates(inplace=True)

In [22]:
len(df)

3015

In [27]:
LINKS

['https://www.forever21.com/us/shop/catalog/category/21men/mens-tees-tanks-graphic',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-bottom',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-jackets-and-coats',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-sweaters',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-sweatshirts-hoodies',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-bottom-swim',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-shirts',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-bottom-denim',
 'https://www.forever21.com/us/shop/catalog/category/21men/mens-bottom-shorts']

In [23]:
df['categoria'].unique()

array(['Graphic Tees for Men', "Men's Shirts",
       "Men's Hoodies + Sweatshirts", "Men's Sweaters & Cardigans",
       "Men's Jackets & Coats", "Men's Bottoms", "Men's Jeans",
       "Men's Button-Down Shirts", 'Mens Shorts', "Men's Swim Trunks"],
      dtype=object)

In [29]:
end_= datetime.datetime.now()

In [24]:
df.to_excel(f'./Salida/F21MEN{fecha}.xlsx')

In [25]:
browser.quit()

In [ ]:
2